In [1]:
import os
from tqdm import tqdm
import sys
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import torch
import re
import numpy as np
import json

e:\AICHCM2024\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
sys.path.append('..')

In [3]:
from config import OCR_PATH, OCR_FEATURES_PATH

In [4]:
ocr_dir = "../" + OCR_PATH
ocr_features_dir = "../" + OCR_FEATURES_PATH

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
model = SentenceTransformer('dangvantuan/vietnamese-embedding', device=device)

In [7]:
def preprocess_text(text: str):
    text = text.lower()
    reg_pattern = r'[^a-z0-9A-Z_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ\s]'
    output = re.sub(reg_pattern, '', text)
    output = output.strip()
    output = " ".join(output.split())
    return output

In [8]:
batch_size = 16
for ocr_file in tqdm(os.listdir(ocr_dir)):
    ocr_file_path = os.path.join(ocr_dir, ocr_file)
    output_path = os.path.join(ocr_features_dir, ocr_file.replace('.json', '.npy'))

    with open(ocr_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    sentences = [preprocess_text(' '.join(sublist)) for sublist in data]
    tokenizers = [tokenize(sent) for sent in sentences]
    truncated_tokenizers = [tokens[:512] for tokens in tokenizers]
    
    video_ocr_features = []

    ocr_features = model.encode(truncated_tokenizers, batch_size=batch_size, convert_to_tensor=True)
    ocr_features /= ocr_features.norm(dim=-1, keepdim=True)

    for idx in range(ocr_features.shape[0]):
        video_ocr_features.append(ocr_features[idx].detach().cpu().numpy().astype(np.float32).flatten())
    
    np.save(output_path, video_ocr_features)

100%|██████████| 726/726 [07:55<00:00,  1.53it/s]
